In [1]:
import dash
import dash_bootstrap_components as dbc
# from dash import dcc
# from dash import html
import dash_core_components as dcc
import dash_html_components as html
from dash import dash_table
import dash_cytoscape as cyto

from dash.dependencies import Input, Output, State


c:\users\goddo\바탕 화면\mind_detector_v2\myvenv\lib\site-packages\dash_bootstrap_components\_table.py:5: UserWarning: 
The dash_html_components package is deprecated. Please replace
`import dash_html_components as html` with `from dash import html`
  import dash_html_components as html
C:\Users\goddo\AppData\Local\Temp/ipykernel_22716/1686502524.py:5: UserWarning: 
The dash_core_components package is deprecated. Please replace
`import dash_core_components as dcc` with `from dash import dcc`
  import dash_core_components as dcc


In [2]:
import pandas as pd
import networkx as nx

from colour import Color
from datetime import datetime
from textwrap import dedent as d
import json

import io
import base64

In [3]:
import plotly.express as px
import plotly.graph_objs as go

# 초기화함수

In [4]:
# def initialize_graph():
global raw, raw_df
raw = round(pd.read_csv('../data/psp_swn_weight_ggg_v2.csv', index_col=0), 3)
raw_v2 = raw.copy()
raw_v2.insert(0, 'attr', raw.columns, allow_duplicates=False)


# 정신질환 질문 속성
global attrSet
attrSet = set(list(raw.columns))
# attrSet.add()

raw_df = pd.DataFrame.from_dict(raw)
raw_df = raw_df.stack().reset_index()
raw_df.columns = ['source', 'target', 'weight']
raw_df.drop(raw_df[abs(raw_df.weight) < 0.01].index, inplace=True)

global basic_graph
basic_graph = nx.from_pandas_edgelist(raw_df, 'source', 'target', ['source', 'target', 'weight'], create_using=nx.Graph())

# correlation to source&tagert

In [5]:
def corr_to_target(df):
    '''
    df(corr) -> weight
    '''
    print('df', df.columns) 
    target_df = pd.DataFrame.from_dict(df)
    target_df = target_df.stack().reset_index()
    target_df.columns = ['source', 'target', 'weight']
    target_df.drop(target_df[abs(target_df.weight) < 0.01].index, inplace=True)
    print('target_df', target_df)
    
    return target_df

# weight to graph

In [6]:
def target_to_graph(target_df):
    '''
    df(weight_df) -> graph
    '''
    basic_graph = nx.from_pandas_edgelist(target_df, 'source', 'target', ['source', 'target', 'weight'], create_using=nx.Graph())
    return basic_graph

# 네트워크 분석 함수

In [7]:
def generate_network_graph(target_df, specific_attr=None):
    attrSet = set(list(target_df.columns))
    
    # networkx layout 배치를위한 중심점 정의
    shells = []

    shell1 = [] #specific_attr을 위한 리스트

    #specific_attr 있으면 추가
    shell1.append(specific_attr)
    shells.append(shell1)


    shell2 = [] #specific_attr을 제외한 요소를 위한 리스트

    # specific_attr와 다른 요소 추가
    for elem in attrSet:
        if elem != specific_attr:
            shell2.append(elem)
    shells.append(shell2)


    G = nx.from_pandas_edgelist(target_df, 'source', 'target', ['source', 'target', 'weight'], create_using=nx.Graph())
 

    if len(shell2) > 1:
        pos = nx.drawing.layout.spring_layout(G)
        print("spring")
    else:
        pos = nx.drawing.layout.shell_layout(G, shells)
        print("shell")

    for node in G.nodes:
        G.nodes[node]['pos'] = list(pos[node])

    # shell2: specific_attr을 제외한 요소를 위한 리스트
    if len(shell2) == 0:
        traceRecode = [] # contains edge_trace, node_trace, middle_node_trace
        node_trace = go.Scatter(x=tuple([1]), y=tuple([1]), text=tuple([str(specific_attr)]), 
                                textposition="bottom center",
                                mode= "markers+text",
                                marker={'size': 25, 'color': 'LightSkyBlue'},
                                opacity = 0)

        node_trace1 = go.Scatter(x=tuple([1]), y=tuple([1]),
                                mode='markers',
                                marker={'size': 25, 'color': 'LightSkyBlue'},
                                opacity=0)
        traceRecode.append(node_trace1)

        figure = {
            "data": traceRecode,
            "layout": go.Layout(title='Network Analaysis', showlegend=False,
                                margin={'b': 40, 'l': 40, 'r': 40, 't': 40},
                                xaxis={'showgrid': False, 'zeroline': False, 'showticklabels': False},
                                yaxis={'showgrid': False, 'zeroline': False, 'showticklabels': False},
                                height=600
                                )}
        return figure
        


    traceRecode = []  # contains edge_trace, node_trace, middle_node_trace
    ############################################################################################################################################################
    pos_colors = list(Color('DarkBlue').range_to(Color('SkyBlue'), 3))
    pos_colors = ['rgb'+str(x.rgb) for x in pos_colors]

    neg_colors = list(Color('OrangeRed').range_to(Color('DarkRed'), 3))
    neg_colors = ['rgb'+str(x.rgb) for x in neg_colors]
    # colors = ['rgb' + str(x.rgb) for x in colors]

    index = 0
    for edge in G.edges:

        x0, y0 = G.nodes[edge[0]]['pos']
        x1, y1 = G.nodes[edge[1]]['pos']
        weight = G.edges[edge]['weight']

        if weight > 0:
            if weight < 0.3:
                selected_color = pos_colors[2]
            elif 0.3 <= weight < 0.6:
                selected_color = pos_colors[1]
            else:
                selected_color = pos_colors[0]
        else:
            if weight <= -0.6:
                selected_color = neg_colors[2]
            elif -0.6 < weight <= 0.3:
                selected_color = neg_colors[1]
            else:
                selected_color = neg_colors[0]

        trace = go.Scatter(x=tuple([x0, x1, None]), y=tuple([y0, y1, None]),
                           mode='lines',
                           line={'width': abs(weight)*25},
                           marker=dict(color=selected_color),
                           line_shape='spline',
                           opacity=1)

        traceRecode.append(trace)
        index = index + 1

    ###############################################################################################################################################################
    node_trace = go.Scatter(x=[], y=[], hovertext=[], text=[], mode='markers+text', textposition="bottom center",
                            hoverinfo="text", marker={'size': 25, 'color': 'LightGreen'})

    index = 0
    for node in G.nodes():
        x, y = G.nodes[node]['pos']
        # ex: Anxiety: 긴장 표시용 일단은 필요 X
        # hovertext = "Attribute Name: " + str(G.nodes[node]['Evaluation item'])
        # node_trace['hovertext'] += tuple([hovertext])

        # text = node1['Account'][index]
        node_trace['x'] += tuple([x])
        node_trace['y'] += tuple([y])
        node_trace['text'] += tuple([node])

        index = index + 1
    traceRecode.append(node_trace)       


    ################################################################################################################################################################
    middle_hover_trace = go.Scatter(x=[], y=[], hovertext=[], mode='markers', hoverinfo="text",
                                    marker={'size': 20, 'color': 'LightSkyBlue'},
                                    opacity=0)

    index = 0
    for edge in G.edges:
        x0, y0 = G.nodes[edge[0]]['pos']
        x1, y1 = G.nodes[edge[1]]['pos']
        hovertext = "From: " + str(G.edges[edge]['source']) +  "<br>" + "To   : " + str(G.edges[edge]['target']) + "<br>" + "correlation: " + str(G.edges[edge]['weight'])

        middle_hover_trace['x'] += tuple([(x0 + x1) / 2])
        middle_hover_trace['y'] += tuple([(y0 + y1) / 2])
        middle_hover_trace['hovertext'] += tuple([hovertext])
        index = index + 1

    traceRecode.append(middle_hover_trace)


    #################################################################################################################################################################
    figure = {
        "data": traceRecode,
        "layout": go.Layout(title='Network Analysis', showlegend=False, hovermode='closest',
                            margin={'b': 40, 'l': 40, 'r': 40, 't': 40},
                            xaxis={'showgrid': False, 'zeroline': False, 'showticklabels': False},
                            yaxis={'showgrid': False, 'zeroline': False, 'showticklabels': False},
                            height=600,
                            clickmode='event+select',

                            )}
    # print(type(figure))
    # print("끝")
    return figure


# 중심성 계산 함수

In [8]:
def network_to_centrality(input_graph, normalized=False):
    
    fig = go.Figure()
    fig.update_layout(
        autosize=True,
        height=600,)
    # degree_centrality
    '''
    input: graph-networkx.Grpah
    output: dict['attribute']=centrality
    '''
    degree_cent = nx.degree_centrality(input_graph)
    degree_cent = dict(sorted(degree_cent.items(), key=lambda item:item[1]))
    fig.add_trace(go.Scatter(x=list(degree_cent.values()), y= list(degree_cent.keys()), mode="lines+markers", name="degree"))
        
    
    # weight_centrality
    weight_cent = {n:0.0 for n in input_graph.nodes()}
    for u, v, d in input_graph.edges(data=True):
        weight_cent[u]+=d['weight']
        weight_cent[v]+=d['weight']
    weight_cent = dict(sorted(weight_cent.items(), key=lambda item:item[1]))
    if normalized==True:
        weighted_sum = sum(weight_cent.values())
        norm_weight_cent = {k:v/weighted_sum for k, v in weight_cent.items()}
        fig.add_trace(go.Scatter(x=list(norm_weight_cent.values()), y= list(norm_weight_cent.keys()), mode="lines+markers", name="weighted degree"))
    else:
        fig.add_trace(go.Scatter(x=list(weight_cent.values()), y= list(weight_cent.keys()), mode="lines+markers", name="weighted degree"))     
                      
                   
    # closeness_centrality
    '''
    input: graph-networkx.Grpah
    output: dict['attribute']=centrality
    '''
    closeness_cent = nx.closeness_centrality(input_graph)
    closeness_cent = dict(sorted(closeness_cent.items(), key=lambda item:item[1]))
    fig.add_trace(go.Scatter(x=list(closeness_cent.values()), y= list(closeness_cent.keys()), mode="lines+markers", name="closeness"))

    
    # betweenness_centrality
    '''
    input: graph-networkx.Grpah
    output: dict['attribute']=centrality
    '''

    between_cent = nx.betweenness_centrality(input_graph, weight='weight', normalized=True)
    between_cent = dict(sorted(between_cent.items(), key=lambda item:item[1]))
    fig.add_trace(go.Scatter(x=list(between_cent.values()), y= list(between_cent.keys()), mode="lines+markers", name="betweenness"))  


    # eigenvector_cntrality
    '''
    input: graph-networkx.Grpah
    output: dict['attribute']=centrality
    '''
    try: 
        eigen_cent = nx.eigenvector_centrality(input_graph, weight='weight')
        eigen_cent = dict(sorted(eigen_cent.items(), key=lambda item:item[1]))
        fig.add_trace(go.Scatter(x=list(eigen_cent.values()), y= list(eigen_cent.keys()), mode="lines+markers", name="eigenvector"))
    except:
        print("eigenvector centrality 오류")
    
    
    # pagerank_centrality
    try:
        pagerank_cent = nx.pagerankG(input_graph, weight='weight')
        pagerank_cent = dict(sorted(between_cent.items(), key=lambda item:item[1]))
        fig.add_trace(go.Scatter(x=list(between_cent.values()), y= list(between_cent.keys()), mode="lines+markers", name="between"))              
    except:
        print("pagerank centrality 오류")
        
    return fig
    

In [9]:
def network_to_weight(input_graph, normalized=False):
    '''
    '''
    weight_cent = {n:0.0 for n in input_graph.nodes()}
    for u, v, d in input_graph.edges(data=True):
        weight_cent[u]+=d['weight']
        weight_cent[v]+=d['weight']
    weight_cent = dict(sorted(weight_cent.items(), key=lambda item:item[1]))
    if normalized==True:
        weighted_sum = sum(weight_cent.values())
        norm_weight_cent = {k:v/weighted_sum for k, v in weight_cent.items()}
        fig = go.Figure(data=[go.Scatter(x=list(norm_weight_cent.values()), y= list(norm_weight_cent.keys()))])
        return fig
    else:
        fig = go.Figure(data=[go.Scatter(x=list(weight_cent.values()), y= list(weight_cent.keys()))])
        return fig


def network_to_degree(input_graph):
    '''
    input: graph-networkx.Grpah
    output: dict['attribute']=centrality
    '''
    degree_cent = nx.degree_centrality(input_graph)
    degree_cent = dict(sorted(degree_cent.items(), key=lambda item:item[1]))
    fig = go.Figure(data=[go.Scatter(x=list(degree_cent.values()), y= list(degree_cent.keys()))])
    
    return fig


def network_to_between(input_graph):
    '''
    input: graph-networkx.Grpah
    output: dict['attribute']=centrality
    '''
    between_cent = nx.betweenness_centrality(input_graph, weight='weight')
    between_cent = dict(sorted(between_cent.items(), key=lambda item:item[1]))
    fig = go.Figure(data=[go.Scatter(x=list(between_cent.values()), y= list(between_cent.keys()))])
    
    return fig

def network_to_eigen(input_graph):
    '''
    input: graph-networkx.Grpah
    output: dict['attribute']=centrality
    '''
    try: 
        eigen_cent = nx.eigenvector_centrality(input_graph, weight='weight', max_iter=0)
    except:
        print("eigenvector centrality 오류")
        eigen_cent = nx.betweenness_centrality(input_graph, weight='weight')
    eigen_cent = dict(sorted(eigen_cent.items(), key=lambda item:item[1]))
    fig = go.Figure(data=[go.Scatter(x=list(eigen_cent.values()), y= list(eigen_cent.keys()))])
    return fig


# 1. Style Sheet

In [10]:
# the style arguments for the sidebar.
SIDEBAR_STYLE = {
    'position': 'fixed',
    'top': 0,
    'left': 0,
    'bottom': 0,
    'width': '20%',
    'padding': '20px 10px',
    'background-color': '#f8f9fa'
}

# the style arguments for the main content page.
CONTENT_STYLE = {
    'margin-left': '25%',
    'margin-right': '5%',
    'padding': '20px 10p'
}

TEXT_STYLE = {
    'textAlign': 'center',
    'color': '#191970'
}

CARD_TEXT_STYLE = {
    'textAlign': 'center',
    'color': '#0074D9'
}


# 2. Control Bar

In [11]:


file_uploads = html.Div([
    dcc.Upload(
        id='upload-file',
        children=html.Div([
            'Attribute Matrix'
        ]),
        style={
            'width': '100%',
            'height': '60px',
            'lineHeight': '60px',
            'borderWidth': '1px',
            'borderStyle': 'dashed',
            'borderRadius': '5px',
            'textAlign': 'center',
            'margin': '10px'
        },
        # Allow multiple files to be uploaded
        multiple=False
    ),
]) 


corr_file_uploads = html.Div([
    dcc.Upload(
        id='upload-corr-file',
        children=html.Div([
            'Correlation Matrix'
        ]),
        style={
            'width': '100%',
            'height': '60px',
            'lineHeight': '60px',
            'borderWidth': '1px',
            'borderStyle': 'dashed',
            'borderRadius': '5px',
            'textAlign': 'center',
            'margin': '10px'
        },
        # Allow multiple files to be uploaded
        multiple=False
    ),
]) 

controls = dbc.FormGroup(
    [
        html.P('Dropdown', style = {
            'textAlign': 'center'
        }),
        dcc.Dropdown(
            id='dropdown',
            options = [ 
                {
                    'label': 'Value One',
                    'value': 'value1'
                }, 
                {
                    'label': 'Value Twoo',
                    'value': 'value2',
                }, 
                {
                    'label': 'Value Three',
                    'value': 'value3'
                },
            ],
            value = ['value1'],
            multi = True
        ),
        html.Br(),
        html.P('Range Slider', style= {
            'textAlign': 'center'
        }),
        dcc.RangeSlider(
            id='range_slider',
            min = 0,
            max = 20,
            step = 0.5,
            value = [5, 15]
        ),
        html.P('Check Box', style={
            'textAlign': 'center'
        }),
        dbc.Card([dbc.Checklist(
            id='check_list',
            options = [
                {
                    'label': 'Value One',
                    'value': 'value1'
                },
                {
                    'label': 'Value Two',
                    'value': 'value2',
                },
                {
                    'label': 'Value Three',
                    'value': 'value3'
                }
            ],
            value = ['value1', 'value2'],
            inline = True
        )]),
        html.Br(),
        html.P('Radio Items', style={
            'textAlign': 'center'
        }),
        dbc.Card([dbc.RadioItems(
            id='radio_items',
            options=[
                {
                    'label': 'Value One',
                    'value': 'value1',
                },
                {
                    'label': 'Value Two',
                    'value': 'value2',
                },
                {
                    'label': 'Value Three',
                    'value': 'value3',
                }
            ],
            value='value1',
            style = { 'margin': 'auto'}
        )]),
        html.Br(),
        dbc.Button(
            id='submit_button',
            n_clicks=0,
            children='Submit',
            color='primary',
            block=True
        ),
    ]
)


sidebar = html.Div(
    [
        file_uploads,
        corr_file_uploads,
        html.Hr(),
        html.H2('Parameters', style=TEXT_STYLE),
        html.Hr(),
        
        controls
    ],
    style=SIDEBAR_STYLE,
)


In [12]:
def download_file(contents, filename):
    content_type, content_string = contents.split(",")
    decoded = base64.b64decode(content_string)

    try:
        if "csv" in filename:
            df = pd.read_csv(io.StringIO(decoded.decode("utf-8")), index_col=0 )
        elif "xls" or "xlsx" in filename:
            df = pd.read_excel(io.BytesIO(decoded), index_col=0)
        elif "txt" or "tsv" in filename:
            df = pd.read_csv(io.StringIO(decoded.decode("utf-8")), delimiter=r"\s+", index_col=0)
    except Exception as e:
        print(e)
        return html.Div(["There was an error processing this file."])

    if 'Unnamed: 0' in df.columns:
        print('Unnamed: 0 삭제')
        del df['Unnamed: 0']
        
    # ---- 알고리즘 시작 ----
    # 결측치 제거
    df = df.dropna(axis=1, how='all')
    df = df.dropna(axis=0, how='any')
    
    print(df.columns)
    return df
    

In [13]:
def attr_to_ggm(contents, filename):
    content_type, content_string = contents.split(",")
    decoded = base64.b64decode(content_string)

    try:
        if "csv" in filename:
            df = pd.read_csv(io.StringIO(decoded.decode("utf-8")))
        elif "xls" or "xlsx" in filename:
            df = pd.read_excel(io.BytesIO(decoded))
        elif "txt" or "tsv" in filename:
            df = pd.read_csv(io.StringIO(decoded.decode("utf-8")), delimiter=r"\s+")
    except Exception as e:
        print(e)
        return html.Div(["There was an error processing this file."])

    # ---- 알고리즘 시작 ----
    # 결측치 제거
    df = df.dropna(axis=1, how='all')
    df = df.dropna(axis=0, how='any')

    # column name 추출
    columnName = list(df.columns.values)

    # best_alpha 계산
    # gamma 값 0.1로 설정해두었으나 변경 가능합니다.
    best_alpha = compute_Best_Alpha(df) 

    # best_alpha 이용해서 QuicGraphicalLasso 계산, model 구축
    estimator = QuicGraphicalLassoEBIC(lam=best_alpha, auto_scale = False, 
                                       verbose=1, tol = 1e-04,
                                       init_method='spearman', path=100, gamma=0.1, 
                                       max_iter=10000, method='quic').fit(df.values)

    # model.precision_ -> corr 변환 후 상삼각행렬 도출
    df = pd.DataFrame(np.triu(-cov2cor(estimator.precision_),1))
    result = df.copy()

    df.columns = columnName
    df.index = columnName

    # 결과 Matrix 파일 저장
    result.to_csv (r'./my_data_frame.csv', index = True, header=True)
    return df

In [14]:
def make_ggm_table(ggm_matrix):
    '''
    ggm_matrix -> table 객체
    '''
    if 'attr' not in ggm_matrix.columns:
         ggm_matrix.insert(0, 'attr', ggm_matrix.columns, allow_duplicates=False)
    return (html.Div(
        [

            dash_table.DataTable(
                columns=[{"name": str(i), "id": str(i)} for i in ggm_matrix.columns],
                data=ggm_matrix.to_dict("records"),
                style_data={
                    'whiteSpace': 'normal',
                    'height': 'auto',
                },
            ),

        ],
    ))

# 3. Content Row

In [15]:
content_first_row = dbc.Row([
    
    dbc.Col(
        dcc.Graph(id='network_graph',
                  figure = generate_network_graph(raw_df)), md=12,
    )
    
])



spring


In [16]:
content_second_row = dbc.Row(
    [
        
        dbc.Col(
            dcc.Graph(id='centrality_graph',
                     figure = network_to_centrality(basic_graph)), 
                     width=12,
                     
        ),

    ],
    # style = { 'height' : '60vh'}
)



eigenvector centrality 오류
pagerank centrality 오류


In [17]:
content_third_row = dbc.Row(
    children = [
        make_ggm_table(raw_v2)
    ],
    id="ggm_table"
)


# content_third_row = dbc.Row(
#     [
#       html.Div(id="ggm_table",
#             [        
#                 dash_table.DataTable(
#                     columns=[{"name": str(i), "id": str(i)} for i in raw_v2.columns],
#                     data=round(raw_v2, 3).to_dict("records"),
#                     style_data={
#                         'whiteSpace': 'normal',
#                         'height': 'auto',
#                     },
#                 )
#             ],
#         ),   
#     ],
# )


In [18]:
content = html.Div(
    [
        html.H2('Network analysis of mental illness evaluation scores', style=TEXT_STYLE),
        html.Hr(),
        content_first_row,
        content_second_row,
        content_third_row,
        # content_fourth_row,
        # content_fifth_row
    ],
    style = CONTENT_STYLE
)

# 4. App 정보

In [19]:
app = dash.Dash(external_stylesheets=[dbc.themes.BOOTSTRAP])
app.layout = html.Div([sidebar, content])
app.title = "Network analysis of mental illness evaluation scores"

# 5. CallBack

In [ ]:
@app.callback(
    [Output('network_graph', 'figure'), Output('centrality_graph', 'figure'), Output('ggm_table', 'children')],
    [Input('upload-file', 'contents'), Input('upload-file', 'filename')]
)
    
def update_graph(contents, filename):

    # corr matrix는 바로 읽음 ( GGM 적용 X )
    df = attr_to_ggm(contents, filename)
       
    # source, target, weight matrix로 변환
    target_df = corr_to_target(df)

    # network analaysis 진행
    network_graph_fig = generate_network_graph(target_df)

    # networkx graph객체로 변환
    input_graph = target_to_graph(target_df)
    
    # 중심성 계산
    centrality_graph_fig = network_to_centrality(input_graph)
    
    return network_graph_fig, centrality_graph_fig, make_ggm_table(df)

In [22]:
@app.callback(
    [Output('network_graph', 'figure'), Output('centrality_graph', 'figure'), Output('ggm_table', 'children')],
    [Input('upload-corr-file', 'contents'), Input('upload-corr-file', 'filename')]
)
    
def corr_update_graph(contents, filename):

    # corr matrix는 바로 읽음 ( GGM 적용 X )
    df = download_file(contents, filename)
       
    # source, target, weight matrix로 변환
    target_df = corr_to_target(df)

    # network analaysis 진행
    network_graph_fig = generate_network_graph(target_df)

    # networkx graph객체로 변환
    input_graph = target_to_graph(target_df)
    
    # 중심성 계산
    centrality_graph_fig = network_to_centrality(input_graph)
    
    return network_graph_fig, centrality_graph_fig, make_ggm_table(df)

In [ ]:

if __name__ == '__main__':

    
    app.run_server(port='8085')

Dash is running on http://127.0.0.1:8085/

 * Serving Flask app '__main__' (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on http://127.0.0.1:8085/ (Press CTRL+C to quit)
127.0.0.1 - - [08/Oct/2021 01:19:24] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [08/Oct/2021 01:19:25] "GET /_dash-layout HTTP/1.1" 200 -
127.0.0.1 - - [08/Oct/2021 01:19:25] "GET /_dash-dependencies HTTP/1.1" 200 -
127.0.0.1 - - [08/Oct/2021 01:19:25] "GET /_dash-component-suites/dash/dcc/async-upload.js HTTP/1.1" 200 -
127.0.0.1 - - [08/Oct/2021 01:19:25] "GET /_dash-component-suites/dash/dcc/async-dropdown.js HTTP/1.1" 200 -
127.0.0.1 - - [08/Oct/2021 01:19:25] "GET /_dash-component-suites/dash/dcc/async-slider.js HTTP/1.1" 200 -


Exception on /_dash-update-component [POST]
Traceback (most recent call last):
  File "c:\users\goddo\바탕 화면\mind_detector_v2\myvenv\lib\site-packages\flask\app.py", line 2070, in wsgi_app
    response = self.full_dispatch_request()
  File "c:\users\goddo\바탕 화면\mind_detector_v2\myvenv\lib\site-packages\flask\app.py", line 1515, in full_dispatch_request
    rv = self.handle_user_exception(e)
  File "c:\users\goddo\바탕 화면\mind_detector_v2\myvenv\lib\site-packages\flask\app.py", line 1513, in full_dispatch_request
    rv = self.dispatch_request()
  File "c:\users\goddo\바탕 화면\mind_detector_v2\myvenv\lib\site-packages\flask\app.py", line 1499, in dispatch_request
    return self.ensure_sync(self.view_functions[rule.endpoint])(**req.view_args)
  File "c:\users\goddo\바탕 화면\mind_detector_v2\myvenv\lib\site-packages\dash\dash.py", line 1336, in dispatch
    response.set_data(func(*args, outputs_list=outputs_list))
  File "c:\users\goddo\바탕 화면\mind_detector_v2\myvenv\lib\site-packages\dash\_callba

127.0.0.1 - - [08/Oct/2021 01:19:25] "POST /_dash-update-component HTTP/1.1" 500 -
127.0.0.1 - - [08/Oct/2021 01:19:25] "GET /_dash-component-suites/dash/dcc/async-plotlyjs.js HTTP/1.1" 200 -
127.0.0.1 - - [08/Oct/2021 01:19:25] "GET /_dash-component-suites/dash/dcc/async-graph.js HTTP/1.1" 200 -
127.0.0.1 - - [08/Oct/2021 01:19:25] "GET /_dash-component-suites/dash/dash_table/async-highlight.js HTTP/1.1" 200 -
127.0.0.1 - - [08/Oct/2021 01:19:25] "GET /_dash-component-suites/dash/dash_table/async-table.js HTTP/1.1" 200 -


Index(['SOM', 'OC', 'IS', 'DEP', 'ANX', 'HOS', 'PHOB', 'PAR', 'PSY',
       'PANSSP_SUM_w0', 'PANSSN_SUM_w0', 'PANSSG_SUM_w0', 'PANSSTotal_w0'],
      dtype='object')
df Index(['SOM', 'OC', 'IS', 'DEP', 'ANX', 'HOS', 'PHOB', 'PAR', 'PSY',
       'PANSSP_SUM_w0', 'PANSSN_SUM_w0', 'PANSSG_SUM_w0', 'PANSSTotal_w0'],
      dtype='object')
target_df             source         target    weight
1              SOM             OC -0.028942
4              SOM            ANX  0.244953
5              SOM            HOS  0.211634
7              SOM            PAR -0.041105
8              SOM            PSY  0.213171
..             ...            ...       ...
156  PANSSTotal_w0            SOM  0.019546
161  PANSSTotal_w0            HOS  0.012545
165  PANSSTotal_w0  PANSSP_SUM_w0  0.724764
166  PANSSTotal_w0  PANSSN_SUM_w0  0.734033
167  PANSSTotal_w0  PANSSG_SUM_w0  0.847971

[108 rows x 3 columns]
spring


127.0.0.1 - - [08/Oct/2021 01:19:32] "POST /_dash-update-component HTTP/1.1" 200 -


eigenvector centrality 오류
pagerank centrality 오류


In [ ]:
@app.callback(
    Output('centrality_graph', 'figure'),
    [Input('upload-file', 'contents'), Input('upload-file', 'filename')])
    
def update_network_graph(contents, filename):
    contents = contents[0]; filename = filename[0]
    
    # attribute matrix를 gausian graphical model matrix로 변환
    ggm_matrix = attr_to_ggm(contents, filename)
       
    # source, target, weight matrix로 변환
    target_df = corr_to_target(ggm_matrix)

    # network analaysis 진행
    network_graph_fig = generate_network_graph(target_df)

    return network_graph_fig

In [ ]:
@app.callback(
    Output('network_graph', 'figure'),
    [Input('upload-file', 'contents'), Input('upload-file', 'filename')])
    
def update_centrality_graph(contents, filename):
    contents = contents[0]; filename = filename[0]
    
    # attribute matrix를 gausian graphical model matrix로 변환
    ggm_matrix = attr_to_ggm(contents, filename)
       
    # source, target, weight matrix로 변환
    target_df = corr_to_target(ggm_matrix)
    
    # networkx graph객체로 변환
    input_graph = target_to_graph(target_df)
    
    # 중심성 계산
    centrality_graph_fig = network_to_centrality(input_graph)
    return centrality_graph_fig

In [ ]:
@app.callback(
    Output('ggm_table', 'children'),
    [Input('upload-file', 'contents'), Input('upload-file', 'filename')])
    
def update_ggm_table(contents, filename):
    contents = contents[0]; filename = filename[0]
    
    # attribute matrix를 gausian graphical model matrix로 변환
    ggm_matrix = attr_to_ggm(contents, filename)
    
    return make_ggm_table(ggm_matrix)

In [ ]:
@app.callback(
    Output('network_graph', 'figure'),
    [Input('upload-corr-file', 'contents'), Input('upload-corr-file', 'filename')])
    
def corr_update_network_graph(contents, filename):

    # corr matrix는 바로 읽음 ( GGM 적용 X )
    df = download_file(contents, filename)
       
    # source, target, weight matrix로 변환
    target_df = corr_to_target(df)

    # network analaysis 진행
    network_graph_fig = generate_network_graph(target_df)

    return network_graph_fig

In [ ]:
@app.callback(
    Output('centrality_graph', 'figure'),
    [Input('upload-corr-file', 'contents'), Input('upload-corr-file', 'filename')])
    
def corr_update_centrality_graph(contents, filename):

    # corr matrix는 바로 읽음 ( GGM 적용 X )
    df = download_file(contents, filename)
       
    # source, target, weight matrix로 변환
    target_df = corr_to_target(df)
    
    # networkx graph객체로 변환
    input_graph = target_to_graph(target_df)
    
    # 중심성 계산
    centrality_graph_fig = network_to_centrality(input_graph)
    return centrality_graph_fig

In [ ]:
@app.callback(
    Output('ggm_table', 'children'),
    [Input('upload-corr-file', 'contents'), Input('upload-corr-file', 'filename')])
    
def corr_update_ggm_table(contents, filename):
    # corr matrix는 바로 읽음 ( GGM 적용 X )
    df = download_file(contents, filename)
    return make_ggm_table(df)

In [ ]:
@app.callback(
    Output('network_graph', 'figure'),
    [Input('upload-file', 'contents'), Input('upload-file', 'filename')])
    
def update_network_graph(contents, filename):
    
    # 파일 입력 부분
    if contents:
        contents = contents[0]
        filename = filename[0]
        print(contents, filename)
        
        content_type, content_string = contents.split(",")
        decoded = base64.b64decode(content_string)
        
        try:
            if "csv" in filename:
                df = pd.read_csv(io.StringIO(decoded.decode("utf-8")))
            elif "xls" or "xlsx" in filename:
                df = pd.read_excel(io.BytesIO(decoded))
            elif "txt" or "tsv" in filename:
                df = pd.read_csv(io.StringIO(decoded.decode("utf-8")), delimiter=r"\s+")
        except Exception as e:
            print(e)
            return html.Div(["There was an error processing this file."])
        
        if 'Unnamed: 0' in test.columns:
            print('Unnamed: 0 삭제')
            del test['Unnamed: 0']
        
        # ---- 알고리즘 시작 ----
        # 결측치 제거
        df = df.dropna(axis=1, how='all')
        df = df.dropna(axis=0, how='any')
        
        # column name 추출
        columnName = list(df.columns.values)    
    
    target_df = corr_to_target(df)
    network_graph_fig = generate_network_graph(target_df)
    return fig

In [ ]:
@app.callback(
    Output('network_graph', 'figure'),
    [Input('submit_button', 'n_clicks')],
    [State('dropdown', 'value'), State('range_slider', 'value'), State('check_list', 'value'),
     State('radio_items', 'value')])
def update_network_graph(n_clicks, dropdown_value, range_slider_value, check_list_value, radio_items_value):

    fig = generate_network_graph()
    return fig

In [ ]:
@app.callback(
    Output('centrality_graph', 'figure'),
    [Input('submit_button', 'n_clicks')],
    [State('dropdown', 'value'), State('range_slider', 'value'), State('check_list', 'value'),
     State('radio_items', 'value')
     ])
def update_centrality(n_clicks, dropdown_value, range_slider_value, check_list_value, radio_items_value):

    fig = generate_network_graph()
    return fig

In [ ]:
@app.callback(
    Output('corr_table', 'figure'),
    [Input('submit_button', 'n_clicks')],
    [State('dropdown', 'value'), State('range_slider', 'value'), State('check_list', 'value'),
     State('radio_items', 'value')
     ])
def update_corr_table(n_clicks, dropdown_value, range_slider_value, check_list_value, radio_items_value):

    fig = generate_network_graph()
    return fig

# 최종. App 실행

In [ ]:
# npn Skeptic Algorithm

# input (x) : raw dataframe

def skeptic(x):
    y = 2 * math.sin(math.pi/6. * x)
    return y

def npn(x):
    df = x.corr(method = "spearman")
    result = df.applymap(skeptic)
    return result

# output (result) : npn skeptic corr dataframe

In [ ]:
# Making Lambda set

# input (df) : npn skeptic corr datatframe의 상삼각행렬

def lamSet(df):
    lamMax = max(np.max(df.values), -np.min(df.values))
    
    lamMin = 0.01 * lamMax
    lamMaxX = math.log(lamMax)
    lamMinX = math.log(lamMin)
    
    lam = np.exp(np.append(np.arange(lamMinX, lamMaxX, step = ((lamMaxX)-(lamMinX))/99), (lamMaxX)))
    
    return lam

# output (lam) : lambda 값 list

In [ ]:
# computing EBIC

# input (model) : QuicGraphicalLassoEBIC model
# input (n) : data 행 개수
# input (p) : data 열 개수
# input (tr) : npn.values (npn(x)로 계산한 값)
# input (gamma) : gamma 값 (= tuning parameter)

def compute_EBIC(model, n, p, tr, gamma):
    
    prec = model.precision_
    E = (np.sum(np.sum(prec != 0, axis=0))-p)
    MLE = (np.log(np.linalg.det(prec))-np.trace(np.dot(tr, prec))) * n
    EBIC = E * 0.5 * np.log(n) + E * gamma * np.log(p) * 2 - MLE
    
    return EBIC

# output (EBIC) : EBIC 계산값 (float)

In [ ]:
# computing Best Alpha

# input (X): raw dataframe
# input (gamma): gamma 값 (= tuning parameter, 0.1로 설정해두었으나 변경 가능합니다.)

def compute_Best_Alpha(X, gamma = 0.1):
    
    tr = npn(X)
    tr = tr.values # npn skeptic
    triu = pd.DataFrame(np.triu(tr, 1)) # 상삼각행렬 구축
    n=X.shape[0] #data 행 개수
    p=X.shape[1] #data 열 개수
    
    lam = lamSet(triu) # lambda list 계산 값 100개
    EBICs = np.zeros(100) # lambda 개수(100개)만큼 EBIC 값 계산해주기 위해 자리 만듦
        
    # EBIC 계산을 위함, lambda 100개에 대하여 계산해야 하므로 100번 반복
    for i in range(100):
        alpha = round(lam[i], 9)
        model = QuicGraphicalLassoEBIC(lam=alpha, auto_scale = True,
                                           verbose=1, tol = 1e-04,
                                           init_method='spearman', path=100, gamma = gamma, 
                                           max_iter=10000, method='quic').fit(X.values)
        
        # lambda 100개에 대해 EBIC 값 계산
        EBIC = compute_EBIC(model, n, p, tr, gamma)
        print("EBIC : "+str(EBIC)+" alpha : "+str(alpha))
        EBICs[i]=EBIC
    
    # EBICs 중 EBIC 값이 가장 작은 lambda 선택해서 best_alpha로 설정 
    # -> best_alpha를 model의 최종 lambda로 설정해서 계산할 것임
    best_idx=np.argmin(EBICs)
    best_alpha=lam[best_idx]
    return float(best_alpha)

In [ ]:
# cov to corr

# input (cov) : cov dataframe (estimator.precision_)

def cov2cor( cov ):
    d = np.sqrt(cov.diagonal())
    cor = ((cov.T/d).T)/d
    cor[ np.diag_indices( cov.shape[0] ) ] = np.ones( cov.shape[0] )
    return cor

# output (cor) : cor dataframe